L'idée : C'est que le client fournit un audio( vocale) le chiffre en cryptage fully homomrophique , et l'envoie au serveur  :

Le serveur ferra deux manipulations intéressantes:
- calcule la durée du silence de l'audio  et la durée du son  
- comparer deux audios pour savoir si deux personnes ont dit la même chose ( plus difficile mais théoriqument faisable)
  |-> Comme pour faire ceci il suffit de transformé les audios en vecteurs , les chiffrées  puis le serveur calcule la distance euclidienne ou la corrélation
  |-> Limite comme c'est principalement des vecteur de flottants , on va devoir manipuler avec un schéma CKKS  , sauf que genère du bruit après chaque multiplication,
  donc va falloir bien manipuler le contexte et choisir ses arguments 


In [3]:
import scipy as sp
import numpy as np
import librosa
import tenseal as ts
import utils
import soundfile as sf


In [5]:
# audio_artifiel_pomme_java, sr = librosa.load('my_data/artificiel_pomme_java.mp3', sr=None)
audio_2s_delay,sd = librosa.load('my_data/python_banane_ia_2sdelay.mp3', sr=None)
print("audio",audio_artifiel_pomme_java)
print("audio shape",audio_artifiel_pomme_java.shape)
# print("sr",sr)
# print(f"Taux d'échantillonnage: {sr}")
# print(f"Nombre d'échantillons: {len(audio_artifiel_pomme_java)}")

bits_scale = 24
context=ts.context(
            ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=16384,
            coeff_mod_bit_sizes=[60, 40, 40, 60]
        )
context.global_scale = pow(2, bits_scale)
context.generate_galois_keys()
my_secret_key = context.serialize(save_secret_key=True)
utils.write_data("Keys_provider/secret_key.txt", my_secret_key)
context.make_context_public()
public_key=context.serialize()
utils.write_data("Keys_provider/public_key.txt", public_key)


audio_2s_delay_encrypted = ts.ckks_vector(context, audio_2s_delay)
audio_artifiel_pomme_java_encrypted = ts.ckks_vector(context, audio_artifiel_pomme_java)
utils.write_data("output_provider/python_banane_ia_2sdelay_encrypted.txt", audio_2s_delay_encrypted.serialize())
utils.write_data("output_provider/artificiel_pomme_java_encrypted.txt", audio_artifiel_pomme_java_encrypted.serialize())


NameError: name 'audio_artifiel_pomme_java' is not defined

In [42]:
loaded_secret_key = utils.read_data("Keys_provider/secret_key.txt")
secret_key = ts.context_from(loaded_secret_key).secret_key()

sigmoid_enc = utils.read_data("uploads/artificiel_pomme_java_manipulated_encrypted.txt")
sigmoid_vector = ts.lazy_ckks_vector_from(sigmoid_enc)
sigmoid_vector.link_context(context)


sigmoid_decrypted = sigmoid_vector.decrypt(secret_key)
print(f"sigmoid_decrypted :",len(sigmoid_decrypted)) 
sf.write("reconstruit.wav", sigmoid_decrypted, 24000)

sigmoid_decrypted : 35712
